<a href="https://colab.research.google.com/github/Enilsonn/BranchAndBoundPO/blob/main/Enilson_Branch_and_Bound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
!pip install mip

In [47]:
from mip import *
from math import ceil, floor
from queue import Queue

In [48]:
def ehInteiro(x, e=1e-6):
    return abs(x - round(x)) < e

In [49]:
def print_solution(model):
    status = model.optimize()
    print("Status ", status)
    print("Solution value =", model.objective_value)
    print("Solution:")
    for v in model.vars:
        if v.x is not None and v.x > 1e-5:
            print(f"{v.name} = {v.x:.2f}")

In [50]:
def solucaoMaxEncontrada(modelos):
    if not modelos:
        return None
    Zd = max(modelos.keys())
    return Zd, modelos[Zd]

In [51]:
"""
    Essas duas funcoes foram criadas pelo professor Teobaldo Bulhoes, estou usando-as aqui
    apenas para debugar o codigo mais a frente
"""

# funcões usadas posteriormente

# resolve o modelo e mostra os valores das variáveis na solução
def solve(model):
  status = model.optimize()

  if status != OptimizationStatus.OPTIMAL:
    return

  print("Status = ", status)
  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [52]:
def lerModelo(path):
    with open(path, "r") as f:
          linhas = f.readlines()

    # numero de variaveis e de restrições
    n_var, n_rest = list(map(int,linhas[0].split()))

    model = Model(sense=MAXIMIZE, solver_name=CBC)

    x = [model.add_var(name=f"x_{i+1}") for i in range(n_var)]

    objective_const = list(map(int, linhas[1].split()))

    objectivo = 0
    for i in range(n_var):
        objectivo += x[i]*objective_const[i]
    model.objective = objectivo

    linha = 2
    for rest in range(n_rest):
        LHS = 0
        const_rest = list(map(int, linhas[linha].split()))

        for i in range(n_var):
            LHS += x[i]*const_rest[i]

        model += LHS <= const_rest[-1]

    return model




In [53]:
def lerModelo(path):
    with open(path, "r") as f:
        linhas = f.readlines()

    # numero de variaveis e de restrições
    n_var, n_rest = list(map(int, linhas[0].split()))

    # coeficientes da função objetivo
    c = list(map(float, linhas[1].split()))

    # restrições
    restricoes = []
    for i in range(n_rest):
        *coef, rhs = map(float, linhas[2 + i].split())
        restricoes.append((coef, rhs))

    # cria o modelo
    model = Model(sense=MAXIMIZE, solver_name=CBC)

    # adiciona variáveis binárias
    x = [model.add_var(var_type=CONTINUOUS, name=f"x{i+1}") for i in range(n_var)]

    # define a função objetivo
    model.objective = xsum(c[i] * x[i] for i in range(n_var))

    # adiciona restrições
    for coef, rhs in restricoes:
        model += xsum(coef[i] * x[i] for i in range(n_var)) <= rhs

    return model

In [54]:
def branchAndBound(model):
    ehMax = model.sense == MAXIMIZE

    # limite primal
    if ehMax:
      Zp = float('-inf') # caso de maximizacao
    else:
       Zp = float('inf') # caso de minimizacao

    fila = Queue()

    model.optimize()

    if model.status != OptimizationStatus.OPTIMAL:
        print("Modelo inicial inviável ou não ótimo.")
        return None

    variaveisAtuais = {v.name: v.x for v in model.vars}
    varFracionaria = None
    for nome, valor in variaveisAtuais.items():
        if not ehInteiro(valor):
            varFracionaria = nome
            break

    # se já é solução inteira
    if not varFracionaria:
        print("Solução já é inteira.")
        print_solution(model)
        return model

    # inicializa fila com o modelo original
    fila.put((model.copy(), varFracionaria))

    # todos os modelos que serao inteiros
    modelosInteiros = {}

    # para nao deixar a arvore se expandir muito
    count = 0

    while not fila.empty():
        modeloAtual, varAtual = fila.get()

        modeloAtual.optimize()

        if modeloAtual.status != OptimizationStatus.OPTIMAL:
            continue

        Zd = modeloAtual.objective_value
        """
            provavelmente nao eh assim que se usa o Zd (limite dual) na teoria, mas foi
            a utilidade que encontrei pra ele aqui. Funcionar como o Z0, Z1, ..., Zn

        """

        """
            caso a solucao encontrada seja viavel, mas nao seja melhor que uma que ja foi encontrada.
            Entao ja mata esse nó e nem continua a execucao
        """
        if Zd is not None:
              if (ehMax and Zd < Zp) or (not ehMax and Zd > Zp):
              # (ehMax and Zd < Zp) -> se for de maximizacao e essa(Zd) solucao inteira eh menor que a melhor ate agora(Zp)
              # (not ehMax and Zd > Zp) - > se for de minimizacao e essa(Zd) solucao inteira eh menor que a melhor ate agora(Zp)
                continue # bound


        variaveis = {v.name: v.x for v in modeloAtual.vars}

        # verifica se a solução é inteira
        varFracionariaAtual = None
        for nome, valor in variaveis.items():
            if not ehInteiro(valor):
                varFracionariaAtual = nome
                break

        # se for inteira
        if not varFracionariaAtual:
            print(f">> Solução inteira encontrada: {modeloAtual.objective_value:.2f}")
            if Zd is not None:
                if (ehMax and Zd > Zp) or (not ehMax and Zd < Zp):
                    # (ehMax and Zd > Zp) -> se for de maximizacao e essa(Zd) solucao inteira eh maior que a melhor ate agora(Zp)
                    # (not ehMax and Zd < Zp) - > se for de minimizacao e essa(Zd) solucao inteira eh menor que a melhor ate agora(Zp)
                    Zp = Zd
                    modelosInteiros[Zp] = modeloAtual.copy()
            continue # bound

        """agora seguiremos com o branch"""

        # cria os dois ramos (esquerda e direita)
        valorFracionario = variaveis[varFracionariaAtual]
        piso = floor(valorFracionario)
        teto = ceil(valorFracionario)

        # ramo inferior: var <= piso
        modeloEsquerda = modeloAtual.copy()
        modeloEsquerda += modeloEsquerda.var_by_name(varFracionariaAtual) <= piso
        fila.put((modeloEsquerda, varFracionariaAtual))

        # ramo superior: var >= teto
        modeloDireita = modeloAtual.copy()
        modeloDireita += modeloDireita.var_by_name(varFracionariaAtual) >= teto
        fila.put((modeloDireita, varFracionariaAtual))

        count += 1
        if count >= 100:
            print("Limite de 100 iterações atingido. Melhor solução encontrada até agora:")
            break

    resultado = solucaoMaxEncontrada(modelosInteiros)
    if resultado:
        print("\nMelhor solução inteira encontrada:")
        print_solution(resultado[1])
        return resultado[1]
    else:
        print("Nenhuma solução inteira viável encontrada.")
        return None


In [55]:
def branchAndBoundBinario(model):
    ehMax = model.sense == MAXIMIZE

    # limite primal
    if ehMax:
      Zp = float('-inf') # caso de maximizacao
    else:
       Zp = float('inf') # caso de minimizacao

    fila = Queue()

    model.optimize()

    if model.status != OptimizationStatus.OPTIMAL:
        print("Modelo inicial inviável ou não ótimo.")
        return None

    variaveisAtuais = {v.name: v.x for v in model.vars}
    varFracionaria = None
    for nome, valor in variaveisAtuais.items():
        if not ehInteiro(valor):
            varFracionaria = nome
            break

    # se já é solução inteira
    if not varFracionaria:
        print("Solução já é inteira.")
        print_solution(model)
        return model

    # inicializa fila com o modelo original
    fila.put((model.copy(), varFracionaria))

    # todos os modelos que serao inteiros
    modelosInteiros = {}

    # para nao deixar a arvore se expandir muito
    count = 0

    while not fila.empty():
        modeloAtual, varAtual = fila.get()

        modeloAtual.optimize()

        if modeloAtual.status != OptimizationStatus.OPTIMAL:
            continue

        Zd = modeloAtual.objective_value
        """
            provavelmente nao eh assim que se usa o Zd (limite dual) na teoria, mas foi
            a utilidade que encontrei pra ele aqui. Funcionar como o Z0, Z1, ..., Zn

        """

        """
            caso a solucao encontrada seja viavel, mas nao seja melhor que uma que ja foi encontrada.
            Entao ja mata esse nó e nem continua a execucao
        """
        if Zd is not None:
              if (ehMax and Zd < Zp) or (not ehMax and Zd > Zp):
              # (ehMax and Zd < Zp) -> se for de maximizacao e essa(Zd) solucao inteira eh menor que a melhor ate agora(Zp)
              # (not ehMax and Zd > Zp) - > se for de minimizacao e essa(Zd) solucao inteira eh menor que a melhor ate agora(Zp)
                continue # bound


        variaveis = {v.name: v.x for v in modeloAtual.vars}

        # verifica se a solução é inteira
        varFracionariaAtual = None
        for nome, valor in variaveis.items():
            if not (valor == 1 or valor == 0):
                varFracionariaAtual = nome
                break

        # se for inteira
        if not varFracionariaAtual:
            print(f">> Solução inteira encontrada: {modeloAtual.objective_value:.2f}")
            if Zd is not None:
                if (ehMax and Zd > Zp) or (not ehMax and Zd < Zp):
                    # (ehMax and Zd > Zp) -> se for de maximizacao e essa(Zd) solucao inteira eh maior que a melhor ate agora(Zp)
                    # (not ehMax and Zd < Zp) - > se for de minimizacao e essa(Zd) solucao inteira eh menor que a melhor ate agora(Zp)
                    Zp = Zd
                    modelosInteiros[Zp] = modeloAtual.copy()
            continue # bound

        """agora seguiremos com o branch"""

        # cria os dois ramos (esquerda e direita)
        valorFracionario = variaveis[varFracionariaAtual]

        # ramo inferior: var <= piso
        modeloEsquerda = modeloAtual.copy()
        modeloEsquerda += modeloEsquerda.var_by_name(varFracionariaAtual) == 0
        fila.put((modeloEsquerda, varFracionariaAtual))

        # ramo superior: var >= teto
        modeloDireita = modeloAtual.copy()
        modeloDireita += modeloDireita.var_by_name(varFracionariaAtual) == 1
        fila.put((modeloDireita, varFracionariaAtual))

        count += 1
        if count >= 100:
            print("Limite de 100 iterações atingido. Melhor solução encontrada até agora:")
            break

    resultado = solucaoMaxEncontrada(modelosInteiros)
    if resultado:
        print("\nMelhor solução inteira encontrada:")
        print_solution(resultado[1])
        return resultado[1]
    else:
        print("Nenhuma solução inteira viável encontrada.")
        return None


In [56]:
def createModel1():
  with open("instance1.txt", "w") as f:
    f.write("""7
    12
    1
    7
    1 2 10
    1 3 8
    1 4 3
    2 3 4
    2 5 4
    3 5 8
    3 6 2
    4 3 3
    4 5 7
    4 6 9
    5 7 10
    6 7 10
    """)
  file_path = "instance1.txt"

  f = open(file_path, "r")
  n = int(f.readline())      # número de vértices (vértices numerados de 1 a n)
  m = int(f.readline())      # número de arcos (arcos numerados de 1 a m)
  source = int(f.readline()) # índice do vértice de origem
  sink = int(f.readline())   # índice do vértice de destino
  arcs = []
  for i in range(m):
      i, j, cap = [int(val) for val in f.readline().split()]
      arcs.append((str(i), str(j), cap))

  # exibe dados da instância
  print(n, m, source, sink)
  for arc in arcs:
    print(arc)
  # implementar modelo
  model = Model(sense= MAXIMIZE, solver_name= CBC)

  # alocando as variaveis
  x = {}
  for (i, j, _) in arcs:
      if i not in x:
          x[i] = {}
      x[i][j] = model.add_var(name=f"x_{i}_{j}")

  # funcao objetivo
  LHS = 0
  for (i, j, _) in arcs:
    if int(j) == sink:
      LHS += x[i][j]
  model.objective = LHS

  print(LHS)

  # restricoes dos vertices
  for (i, j, c) in arcs:
    model += x[i][j] >= 0
    model += x[i][j] <= c

  # restricoes dos vertices
  for v in range(1, n+1):

    if v == source or v == sink: continue

    entra = 0
    for (i, j, _) in arcs:
      if int(j) == v:
        entra += x[i][j]

    sai = 0
    for (i, j, _) in arcs:
      if int(i) == v:
        sai += x[i][j]

    #restricao
    model += (entra - sai) == 0

  return model

In [58]:
def createModel2():
  model = Model(sense=MAXIMIZE, solver_name=CBC)
  portas = ["Porta_de_madeira", "Porta_de_aluminio"]

  x = {
      "Porta_de_madeira": model.add_var(name="Porta_de_madeira"),
      "Porta_de_aluminio": model.add_var(name="Porta_de_aluminio")
  }




  model.objective = 4*x[portas[0]] + 6*x[portas[1]]

  model += 1.5*x[portas[0]] + 4*x[portas[1]] <= 24
  model += 3*x[portas[0]] + 1.5*x[portas[1]] <= 21
  model += x[portas[0]] + x[portas[1]] <= 8

  return model

In [57]:
branchAndBound(createModel1())

7 12 1 7
('1', '2', 10)
('1', '3', 8)
('1', '4', 3)
('2', '3', 4)
('2', '5', 4)
('3', '5', 8)
('3', '6', 2)
('4', '3', 3)
('4', '5', 7)
('4', '6', 9)
('5', '7', 10)
('6', '7', 10)
+ x_5_7 + x_6_7 
Solução já é inteira.
Status  OptimizationStatus.OPTIMAL
Solution value = 15.0
Solution:
x_1_2 = 4.00
x_1_3 = 8.00
x_1_4 = 3.00
x_2_5 = 4.00
x_3_5 = 6.00
x_3_6 = 2.00
x_4_6 = 3.00
x_5_7 = 10.00
x_6_7 = 5.00


In [59]:
branchAndBound(createModel2())

>> Solução inteira encontrada: 40.00

Melhor solução inteira encontrada:
Status  OptimizationStatus.OPTIMAL
Solution value = 40.0
Solution:
Porta_de_madeira = 4.00
Porta_de_aluminio = 4.00


In [62]:
branchAndBound(lerModelo("teste1.txt"))

>> Solução inteira encontrada: 30.00
>> Solução inteira encontrada: 30.00
>> Solução inteira encontrada: 30.00
>> Solução inteira encontrada: 30.00

Melhor solução inteira encontrada:
Status  OptimizationStatus.OPTIMAL
Solution value = 30.0
Solution:
x5 = 2.00
x6 = 1.00


In [63]:
branchAndBoundBinario(lerModelo("teste1.txt"))

>> Solução inteira encontrada: 8.00
>> Solução inteira encontrada: 9.00
>> Solução inteira encontrada: 13.00
>> Solução inteira encontrada: 14.00
>> Solução inteira encontrada: 15.00
>> Solução inteira encontrada: 16.00
>> Solução inteira encontrada: 17.00
>> Solução inteira encontrada: 18.00
>> Solução inteira encontrada: 18.00
>> Solução inteira encontrada: 20.00
>> Solução inteira encontrada: 20.00
>> Solução inteira encontrada: 20.00

Melhor solução inteira encontrada:
Status  OptimizationStatus.OPTIMAL
Solution value = 20.0
Solution:
x2 = 1.00
x5 = 1.00


In [65]:
branchAndBoundBinario(lerModelo("teste2.txt"))

>> Solução inteira encontrada: 12.00
>> Solução inteira encontrada: 19.00
>> Solução inteira encontrada: 24.00
>> Solução inteira encontrada: 24.00

Melhor solução inteira encontrada:
Status  OptimizationStatus.OPTIMAL
Solution value = 24.0
Solution:
x6 = 1.00
x7 = 1.00
x9 = 1.00


In [66]:
branchAndBoundBinario(lerModelo("teste3.txt"))

>> Solução inteira encontrada: 9.00
>> Solução inteira encontrada: 16.00
>> Solução inteira encontrada: 17.00
>> Solução inteira encontrada: 17.00
>> Solução inteira encontrada: 18.00
>> Solução inteira encontrada: 19.00

Melhor solução inteira encontrada:
Status  OptimizationStatus.OPTIMAL
Solution value = 19.0
Solution:
x3 = 1.00
x7 = 1.00


In [67]:
branchAndBoundBinario(lerModelo("teste4.txt"))

>> Solução inteira encontrada: 9.00
>> Solução inteira encontrada: 9.00
>> Solução inteira encontrada: 9.00
>> Solução inteira encontrada: 10.00

Melhor solução inteira encontrada:
Status  OptimizationStatus.OPTIMAL
Solution value = 10.0
Solution:
x3 = 1.00
